# The Vintage Haven (RFM Customer Segmentation)
Dataset Source: Chen, D. (2015). Online Retail [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C5BW33.  
Author: Klaudia Rapa   
Date: 24/11/2025

This is a transactional data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. 

## 1. Import Packages and Data

In [5]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rapidfuzz import fuzz
from sklearn.cluster import AgglomerativeClustering

In [ ]:
# Import data
raw_data = pd.read_excel("/Users/klaudiarapa/Desktop/RFM Customer Segmentation/Online Retail.xlsx")

## 2. Data Cleaning

The first step is to get an overview and clean the dataset.

We aim to understand:  
- Which columns are present and their data types  
- The size of the dataset (number of rows and columns)  

Next, we will assess data quality by checking for missing values and inspecting a small sample of the dataset.

In [ ]:
raw_data.info()

In [ ]:
raw_data.head()

The raw dataset contains **541,909 rows and 8 columns**.  
The data types are appropriate and the dataset is ready for further analysis.

#### **Missing data exploration** 

We examined the dataset for missing values to assess data completeness and identify potential issues.

In [ ]:
# Print count of missing data per column
raw_data.isnull().sum()

There is **135,080 rows with missing Customer ID**. Since these records cannot be used for customer segmentation, we will drop them.

A potential solution could be to assign Customer IDs using other data sources, such as matching by InvoiceID, if feasible.

In [ ]:
# Drop rows with missing Customer ID
raw_data = raw_data.dropna(subset=["CustomerID"])

In [ ]:
# Missing data exploration
raw_data.isnull().sum()

After removing the rows with missing Customer ID, there are no remaining missing values in the dataset.
It’s worth noting that some of the excluded rows also had missing **Description** values.

#### **Cancelled Invoices**

The dataset documentation indicates that **some invoices were cancelled**, which can be identified by invoice numbers starting with "C".
These records should be excluded from the RFM analysis because **cancelled transactions do not reflect actual customer purchasing behavior**.

In [ ]:
# Printing rows with cancelled invoices
raw_data[raw_data["InvoiceNo"].astype(str).str.startswith("C")]

In [ ]:
# Removing the cancelled invoices
raw_data = raw_data[~raw_data["InvoiceNo"].astype(str).str.startswith("C")]

#### **Duplicated records**

**Duplicate records were identified** in the dataset and removed.
Keeping duplicates could lead to artificially inflating purchase frequency and monetary values.

In [ ]:
# -----------------------------------------------------------
# Step 1: Identify and display duplicate rows before dropping
# -----------------------------------------------------------

# Mark all duplicates (keep=False marks all occurrences)
duplicate_rows = raw_data[raw_data.duplicated(keep=False)]

# Show the total number of duplicate rows
print(f"Total duplicate rows found: {duplicate_rows.shape[0]}")

In [ ]:
# Drop duplicates
clean_data = raw_data.drop_duplicates()

#### **How did the cleaning affect the data shape?**

In [ ]:
print("Shape after cleaning:", clean_data.shape)

After cleaning, the dataset contains **392 732 rows and 8 columns**, which represents **72.47% of the original dataset**.
During the cleaning process, we excluded:

- **135,080 rows** with missing Customer ID
- **8,908 rows** corresponding to cancelled invoices (InvoiceNo starting with "C")
- **10,001** duplicate rows

The largest portion of removed data is due to missing Customer IDs, which warrants further investigation to understand the underlying cause.

## 3. Data Exploration

### **Quantity**

**To better understand customer purchasing behavior**, we first examine the Quantity of items ordered.
Using descriptive statistics provides insight into **the typical order size, the spread of values, and the presence of extreme orders**.  

This helps us identify natural breaks in the data, justify segmentation, and detect any unusual or outlier orders.

In [ ]:
# Calculating statistics for Quantity
clean_data['Quantity'].describe()

In [ ]:
# Percentiles
percentiles = [0, 0.25, 0.5, 0.75, 0.9, 0.99, 1.0]
clean_data['Quantity'].quantile(percentiles)

The distribution of order quantities is highly skewed: the majority of customers place relatively small orders (90% are 24 units or fewer), while only 1% of orders exceed 120 units. This supports our decision to segment customers into three groups — Individuals (<20 units), Small B2B (20–100 units), and Wholesalers (>100 units) — so that the RFM analysis fairly reflects different purchasing behaviors. Based on data exploration and discussions with stakeholders, we identified three distinct customer groups:

1. **Individual customers:** purchase fewer than 20 units
2. **Small B2B businesses:** purchase between 20 and 100 units
3. **Wholesalers:** purchase more than 100 units

Since the RFM method can be biased when applied to customers with such different purchasing patterns - favoring wholesalers due to their larger volumes - **we decided to segment the customers into these groups** before calculating RFM scores.

**Mistake:** first I classified customers just based on the Quantity - which led to assigning different Customer Types to the same customer. Identified that mistake during further data exploration and fixed that by assigning Customer Types based on average Quantity per Customer. 

clean_data['Customer Type'] = np.where(
    clean_data['Quantity'] >= 100, 'Wholesalers',
    np.where(clean_data['Quantity'] >= 20, 'Small B2B', 'Individual')
)

In [ ]:
customer_quantity = clean_data.groupby('CustomerID')['Quantity'].agg(['sum','mean','max']).reset_index()
customer_quantity.head()

In [ ]:
def classify_customer(avg_qty):
    if avg_qty < 20:
        return "Individual"
    elif avg_qty < 100:
        return "Small B2B"
    else:
        return "Wholesaler"

customer_quantity['Customer Type'] = customer_quantity['mean'].apply(classify_customer)

In [ ]:
clean_data = clean_data.merge(customer_quantity[['CustomerID','Customer Type']], on='CustomerID', how='left')

In [ ]:
clean_data

In [ ]:
# Count number of orders per customer group
group_counts = clean_data['Customer Type'].value_counts().sort_index()
print("Number of orders per group:")
print(group_counts)

# Optional: show percent of total for each group
group_percent = clean_data['Customer Type'].value_counts(normalize=True).sort_index() * 100
print("\nPercentage of orders per group:")
print(group_percent)

# Optional: visualize distribution per group
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.countplot(data=clean_data, x='Customer Type', order=['Individual','Small B2B','Wholesaler'])
plt.title('Number of Orders per Customer Group')
plt.ylabel('Number of Orders')
plt.show()

In [ ]:
# Filter Wholesalers
wholesalers = clean_data[clean_data["Customer Type"] == "Wholesaler"]

# Count number of orders per Wholesaler
orders_per_wholesaler = wholesalers['CustomerID'].value_counts().sort_values(ascending=False)

print("Number of orders per Wholesaler:")
print(orders_per_wholesaler)

# Optional: visualize
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,5))
sns.histplot(orders_per_wholesaler, bins=30, kde=False)
plt.xlabel('Number of Orders per Wholesaler')
plt.ylabel('Number of Wholesalers')
plt.title('Order Frequency of Wholesalers')
plt.show()


### **Unit Price**

To understand the value of individual products, we examine the UnitPrice column.

Descriptive statistics help us see **the typical price, the range of prices, and any extremely high or low values**.
This information is useful for identifying pricing patterns, detecting outliers, and supporting customer or product segmentation.

In [ ]:
clean_data['UnitPrice'].describe()

The unit price distribution shows that **most products are priced quite low, but there are a few extremely expensive items**:

The average unit price is around 3.13, but the standard deviation is very high (22.24), indicating wide variability.

Most products are inexpensive:
- 25% of orders are <= 1.25
- 50% (median) are <= 1.95
- 75% are <= 3.75

**The minimum price is 0, and the maximum price is 8,142.75, showing a few extreme outliers.**

The majority of orders are for low-priced items, while a small number of very expensive products significantly increase the mean and standard deviation. **This skewed distribution suggests that when analyzing revenue or monetary value, extreme prices may dominate unless handled carefully.**

In [ ]:
g = sns.FacetGrid(
    clean_data, 
    col="Customer Type", 
    col_wrap=3, 
    height=4,
    sharex=False,
    sharey=False      # <- THIS gives each facet its own scale
)

g.map_dataframe(
    sns.scatterplot, 
    x="Quantity", 
    y="UnitPrice", 
    alpha=0.6
)

g.fig.suptitle("Unit Price vs Quantity by Customer Type (Individual Scales)", y=1.05)
plt.show()

In [ ]:
clean_data[clean_data['UnitPrice'] > 1000]

The graphs reveal that **some unit prices are extremely high**. Examining these transactions shows that the *Description* is often vague, such as "Manual" or "DOTCOM POSTAGE", making it difficult to determine whether these values are errors. Because we cannot verify them, **we will retain these data points**, but we should keep in mind that such extreme values exist and **may influence certain analyses**.

### **Country**

The store is primarily located in the UK, which is also home to the majority of its customers; however, customers are spread across 37 different countries.

In [ ]:
clean_data["Country"].describe()

Let’s take a closer look at the geographic distribution of orders across countries.

In [ ]:
# Count number of orders per country
country_orders = clean_data.groupby("Country", as_index=False)["InvoiceNo"].nunique()

# Rename column for clarity
country_orders.rename(columns={"InvoiceNo": "Number of Orders"}, inplace=True)

# Sort by number of orders descending
country_orders = country_orders.sort_values(by="Number of Orders", ascending=False)

# Plot
fig, ax = plt.subplots(figsize=(12,6))
sns.barplot(data=country_orders, x="Country", y="Number of Orders", ax=ax)

# Rotate x-axis labels
ax.tick_params(axis='x', rotation=90)

plt.title("Number of Orders by Country (Sorted)")
plt.tight_layout()
plt.show()



The main market is the UK, with 16,000 orders. We now want to explore which other countries contribute most to sales.

In [ ]:
# Count number of orders per country
country_orders = clean_data.groupby("Country", as_index=False)["InvoiceNo"].nunique()

# Rename column for clarity
country_orders.rename(columns={"InvoiceNo": "Number of Orders"}, inplace=True)

# Sort by number of orders descending
country_orders = country_orders.sort_values(by="Number of Orders", ascending=False)

#Exclude UK 
country_orders = country_orders[country_orders["Country"] != "United Kingdom"]

# Plot
fig, ax = plt.subplots(figsize=(12,6))
sns.barplot(data=country_orders, x="Country", y="Number of Orders", ax=ax)

# Rotate x-axis labels
ax.tick_params(axis='x', rotation=90)

plt.title("Number of Orders by Country (Sorted)")
plt.tight_layout()
plt.show()


Excluding the UK, the top three countries by number of orders are Germany, France, and Ireland.

### **Product (Description)**

To understand customer preferences and identify high-demand items, we examine the Description column, which lists all products in the dataset. By analyzing which products are ordered most frequently or in the highest quantities, we can uncover best-selling items and gain insights for marketing and promotional strategies.

In [ ]:
clean_data["Description"].describe()

The product range is very wide and various. There are 3877 unique product descriptions. The most popular 50 products are displayed on the graph below. 

In [ ]:
# Aggregate total Quantity per Product
product_agg = clean_data.groupby("Description", as_index=False)["Quantity"].sum()

# Sort by Quantity descending
product_agg = product_agg.sort_values(by="Quantity", ascending=False).head(50)

fig, ax = plt.subplots(figsize=(10,6))
sns.barplot(data=product_agg, x="Description", y="Quantity", ax=ax)

# Rotate x-axis labels
ax.tick_params(axis='x', rotation=90)

plt.title("Total Quantity by Description (Sorted)")
plt.tight_layout()
plt.show()

Automatic Merging of Similar Product Descriptions

The dataset contains 3 877 unique product descriptions, some of which are duplicates or slightly different spellings of the same item. To clean the data for analysis, we applied an automated string similarity and clustering approach:

- Compute pairwise similarity of all unique product descriptions using RapidFuzz with token-based string comparison.
- Convert similarity to distance for clustering.
- Cluster similar descriptions using agglomerative clustering (sklearn), so that descriptions with high similarity (e.g., >90%) are grouped together.
- Assign a representative name to each cluster, typically the most frequent description within the cluster.
- Map all orders to the cleaned product descriptions to create a consistent Description_Clean column.

This approach ensures that near-duplicate product names are merged automatically, reducing noise in the analysis of the most popular products, without manually inspecting thousands of entries.

In [ ]:
# Get unique product descriptions
products = clean_data['Description'].unique()

# Build a similarity matrix (token_sort_ratio)
n = len(products)
similarity_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        score = fuzz.token_sort_ratio(products[i], products[j])
        similarity_matrix[i, j] = score
        similarity_matrix[j, i] = score

# Convert similarity to distance (needed for clustering)
distance_matrix = 100 - similarity_matrix

# Cluster similar products
clustering = AgglomerativeClustering(
    n_clusters=None,            # let threshold decide
    metric='precomputed',       # instead of affinity
    linkage='complete',
    distance_threshold=10       # adjust threshold (10 = 90% similarity)
)
labels = clustering.fit_predict(distance_matrix)


# Map original descriptions to cluster labels
product_clusters = pd.DataFrame({'Description': products, 'Cluster': labels})

# Create a representative name for each cluster (e.g., most frequent description)
cluster_map = product_clusters.groupby('Cluster')['Description'].agg(lambda x: x.value_counts().index[0]).to_dict()

# Map original data to cleaned description
clean_data['Description_Clean'] = clean_data['Description'].map(lambda x: cluster_map[product_clusters[product_clusters['Description']==x]['Cluster'].values[0]])


In [ ]:
clean_data["Description_Clean"].describe()

The number of unique products was reduced from 3877 to 3608. We are going to keep both columns in the dataset to further manually validate it in PowerBI. 

### **Invoice Date**

We will examine how the number of orders changes over time to identify any visible trends or seasonal patterns. By breaking down the data by customer type, we can also observe how purchasing behavior differs among Individuals, Small B2B businesses, and Wholesalers. This analysis will help us understand temporal dynamics in sales and guide targeted strategies for each customer segment.

In [ ]:
# Prepare the aggregated data
orders_per_day_type = (
    clean_data.groupby([clean_data['InvoiceDate'].dt.month, 'Customer Type'])['InvoiceNo']
    .nunique()
    .reset_index()
)
orders_per_day_type.rename(columns={'InvoiceNo': 'Number of Orders', 'InvoiceDate': 'Date'}, inplace=True)

# Create FacetGrid
g = sns.FacetGrid(orders_per_day_type, col="Customer Type", col_wrap=3, height=4, sharey=False)
g.map_dataframe(sns.lineplot, x="Date", y="Number of Orders")
g.set_titles("{col_name}")
g.set_axis_labels("Date", "Number of Orders")
for ax in g.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


For all customer groups, November is the busiest month—probably because of the Christmas season and people buying gifts. Orders gradually pick up throughout the year, with January and February being the slowest. This tells us that the company should start stocking up and ramping up marketing in September and October to be ready for the November–December rush.

# 4. Feature Engineering

To prepare the dataset for advanced analysis, we created several additional columns:

- Total Transaction Value (TotalPrice) – calculated as Quantity × UnitPrice. This represents the monetary value of each order and is essential for monetary-based metrics in RFM analysis and revenue reporting.
- Reference Date (ref_date) – a fixed date used to calculate recency for each customer. By having a consistent reference point, we can accurately measure the number of days since a customer’s last purchase.
- Date and Time Columns – derived from the InvoiceDate, including year, month, and day. These allow us to: Analyze seasonal trends and monthly/weekly patterns in orders. Segment transactions by time of day or day of week for more granular insights.


These additional columns enrich the dataset, enabling more precise customer segmentation, time-based analysis, and monetary calculations. Without them, it would be difficult to calculate recency, track revenue contributions, or uncover temporal trends in customer behavior.

In [ ]:
# Create total transaction value
clean_data['TotalPrice'] = clean_data['Quantity'] * clean_data['UnitPrice']

In [ ]:
# Create Reference Date (day after last transaction)
ref_date = clean_data['InvoiceDate'].max() + pd.Timedelta(days=1)
print("Reference date:", ref_date)

In [ ]:
# Divide "Invoice Date" into two separate columns
clean_data["Date"] = clean_data['InvoiceDate'].dt.date
clean_data["Time"] = clean_data['InvoiceDate'].dt.time

In [ ]:
clean_data.head()

In [ ]:
clean_data.info()

The final dataset after cleaning and adding new columns, we have 392 732 rows and 13 columns.

# 5. RFM Calculation

To better understand customer behavior and segment the customer base, we apply **RFM (Recency, Frequency, Monetary)** analysis. This method quantifies:

- **Recency (R)**: how recently a customer made a purchase, calculated as the number of days since their last order relative to a reference date.
- **Frequency (F)**: how often a customer places orders, measured by the number of unique invoices.
- **Monetary (M)**: the total value of a customer’s purchases, summed from TotalPrice.

Because our dataset includes Individuals, Small B2B, and Wholesalers, RFM metrics can be biased if calculated across all customers together - for example, wholesalers naturally have higher frequency and monetary values. By segmenting customers into these groups, we ensure the RFM analysis fairly reflects different purchasing behaviors and provides meaningful insights for each type.

Before scoring, we calculate for each CustomerID:

- Recency: Days since the last purchase relative to a chosen reference date.
- Frequency: Number of unique invoices per customer.
- Monetary: Total spend (sum of TotalPrice).

This produces one RFM record per customer.

In [ ]:
rfm = clean_data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (ref_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                              # Frequency
    'TotalPrice': 'sum'                                  # Monetary
}).reset_index()

In [ ]:
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)

rfm.head()

In [ ]:
# Keep one Customer Type per CustomerID (assuming each customer has a consistent type)
customer_types = clean_data[['CustomerID', 'Customer Type']].drop_duplicates()

# Merge into rfm
rfm = rfm.merge(customer_types, on='CustomerID', how='left')

rfm.head()

### RFM Scoring (1–5 scale)

To transform raw RFM values into categorical scores, we assign each customer a value from 1 to 5, where:
- 5 = best performance
- 1 = weakest performance

**Why scoring is done within each customer type?**

Wholesalers purchase more frequently and in larger amounts than Individual customers.
To avoid biased scoring, we compute quantile-based scores separately within:
- Individual customers
- Small B2B customers
- Wholesalers

This ensures that each segment is compared fairly within its peer group.

In [ ]:
# Function to safely assign quintiles within each group

def score_quintile_rank(x, ascending=True):
    # percentile rank (0–1)
    pct = x.rank(method="average", pct=True)

    # if lower values should get higher score (Recency)
    if ascending:
        pct = 1 - pct

    # convert percentile into 5 bins
    return (pct * 5).apply(lambda v: min(max(int(np.ceil(v)), 1), 5))


In [ ]:
# Recency: lower is better
rfm['R_score'] = (
    rfm.groupby('Customer Type')['Recency']
       .transform(lambda x: score_quintile_rank(x, ascending=False))
)

In [ ]:
# Frequency: higher is better, handle duplicates
rfm['F_score'] = (
    rfm.groupby('Customer Type')['Frequency']
       .transform(lambda x: score_quintile_rank(x, ascending=True))
)

In [ ]:
# Monetary: higher is better
rfm['M_score'] = (
    rfm.groupby('Customer Type')['Monetary']
       .transform(lambda x: score_quintile_rank(x, ascending=True))
)

In [ ]:
rfm.head()

# 6. Customer Segment Mapping

After calculating:
- R_score (1–5)
- F_score (1–5)
- M_score (1–5)

We assign each customer a marketing segment based on classical RFM rules.

- **Champions** - High R, F, M → most loyal, highest spenders, purchase very recently.
- **Loyal** - Buy frequently and repeatedly; stable purchasing patterns.
- **Potential Loyalists** -	Growing customers showing positive recency and frequency trends.
- **Recent Customers** - Recently acquired; good recency but low frequency so far.
- **At Risk but Valuable** - High historical frequency/monetary value but poor recency → high churn risk.
- **At Risk / Lost** -	Low recency and low frequency; likely inactive or churned.
- **Others Customers** - who do not fit cleanly into one of the stronger behavioural patterns.

These segments translate raw purchasing data into clear, actionable customer profiles.

In [ ]:
# Custom mapping based on classical RFM marketing rules
def segment_customer(df):

    R = df['R_score']
    F = df['F_score']
    M = df['M_score']

    # 1. Champions — best in all dimensions
    if R >= 4 and F >= 4 and M >= 4:
        return 'Champions'

    # 2. Loyal — buy often, recent or consistent
    elif R >= 4 and F >= 3:
        return 'Loyal'

    # 3. Potential Loyalist — fairly recent + decent frequency
    elif R >= 3 and F >= 3:
        return 'Potential Loyalist'

    # 4. Recent Customers — good recency but low frequency
    elif R >= 4 and F <= 2:
        return 'Recent Customers'

    # 5. At Risk but Valuable — low recency but high frequency/monetary earlier
    elif R <= 2 and (F >= 4 or M >= 4):
        return 'At Risk but Valuable'

    # 6. At Risk / Lost — weak recency and weak frequency
    elif R <= 2 and F <= 2:
        return 'At Risk / Lost'

    # 7. Remaining customers
    else:
        return 'Others'


In [ ]:
rfm['Segment'] = rfm.apply(segment_customer, axis=1)

In [ ]:
rfm.groupby('Segment')['CustomerID'].count()

Finally, we merge the RFM data back into the full transaction dataset so that every transaction row contains:
- R, F, M values
- RFM scores
- Assigned Segment

This allows further analysis such as:
- Revenue per segment
- Product preferences by segment
- Segment-based customer lifetime value
- Targeted marketing strategies

In [ ]:
# Merge RFM values and segment back to the full transaction table
clean_data_rfm = clean_data.merge(
    rfm[['CustomerID',
         'Recency', 'Frequency', 'Monetary',
         'R_score', 'F_score', 'M_score', 'Segment']],
    on='CustomerID',
    how='left'
)

In [ ]:
clean_data_rfm.info()

# 7. Export Clean Files For Power BI

In [ ]:
rfm.to_csv("/Users/klaudiarapa/Desktop/RFM Customer Segmentation/RFM_Segments.csv", index=False)
print("RFM file exported: RFM_Segments.csv")

clean_data_rfm.to_csv("/Users/klaudiarapa/Desktop/RFM Customer Segmentation/Clean_Transactions.csv", index=False)
print("Cleaned transaction file exported: Clean_Transactions.csv")